In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns


In [ ]:
!pip install datasets

In [ ]:
foundations = ['care','harm','fairness','cheating','loyalty','betrayal','authority','subversion','sanctity','degradation']

## Downloading the datasets to be used as locals files 

In [ ]:
from datasets import load_dataset

dataset = load_dataset("USC-MOLA-Lab/MFRC")
df_annotated = pd.DataFrame(dataset["train"])
idx = []
#found = []

for i in range(len(df_annotated)):

  foundation = df_annotated["annotation"].iloc[i]
  if foundation.lower() in foundations:
    pass 
  else:
    idx.append(i)
idx_ = df_annotated.iloc[idx].index.values    
df_anno = df_annotated.drop(index = idx_)
df_annotated["annotation"].value_counts()

In [ ]:
!gdown --id 1N0U-i4PxT_DmTru0r7T5po-9f9cwGgB8 -O RedditComments2.csv
!gdown --id 1dlV7huXQfim8brL7LIBy-M2kFR63P2Oh -O Reddit_metadata2.csv
!gdown --id 1vT1g4Crj6s6z2YNjSyPx5Fvz5KTpBYSB -O annottated_reddit.csv

In [ ]:
import re
def remove_numbers(text):
  result = re.sub(r'\d+', '', text)
  return result

In [39]:
df_comments= pd.read_csv("RedditComments2.csv")
df_meta = pd.read_csv("Reddit_metadata2.csv")
df_comments.drop_duplicates(inplace= True)
df_comments.dropna(inplace=True)
df_meta.drop_duplicates(inplace= True)
df_meta.dropna(inplace=True)
df_comments.head()
df_annotated = pd.read_csv("annottated_reddit.csv")
df_annotated.drop_duplicates(inplace= True)
df_annotated.dropna(inplace=True)

In [ ]:
df_comments.head()

,Unnamed: 0,comment_id,comment_parent_id,comment_body,comment_link_id,comment_important_words
0,0,gk54wx6,t3_l2d2ba,I have an adult brother with Down Syndrome who...,t3_l2d2ba,"['adult', 'brother', 'syndrom', 'live', 'indep..."
1,1,gk4u2ho,t3_l2d2ba,>am I just overthinking the entire situation?\...,t3_l2d2ba,"['overthink', 'entir', 'situat', 'think', '99'..."
2,2,gk5281a,t3_l2d2ba,I mentored an adult with Down Syndrome. He sp...,t3_l2d2ba,"['mentor', 'adult', 'syndrom', 'spoke', 'well'..."
3,3,gk4nwfl,t3_l2d2ba,"No harm in checking, he could have been practi...",t3_l2d2ba,"['harm', 'check', 'could', 'practic', 'crane',..."
4,4,gk4ovgd,t3_l2d2ba,"Be a good'un and go ask him, it's only fair be...",t3_l2d2ba,"['good', 'un', 'go', 'ask', 'fair', 'may', 'ac..."


In [40]:
df_comments["comment_body"] = np.array([t.lower() for t in df_comments["comment_body"]])

In [32]:
df_annotated.head()

,text,subreddit,bucket,annotator,annotation,confidence
0,MLP doesn't need to wait for a referendum to b...,europe,French politics,annotator00,Authority,Not Confident
1,Or - or - assclowns like Le Pen and Farage cou...,worldnews,French politics,annotator02,Equality,Confident
2,Congratulations on your victory Macron voters....,worldnews,French politics,annotator00,Care,Not Confident
3,The German Constitution did not let Hitler bec...,neoliberal,French politics,annotator02,Equality,Somewhat Confident
4,So Republicans really are for liberal policies...,politics,US Politics,annotator03,Equality,Confident


In [ ]:
#from nltk.stem.porter import *
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk
nltk.download('stopwords')
  

def tweet_to_words(tweet):

  ''' Convert tweet text into a sequence of words '''
  
  # convert to lowercase
  text = tweet.lower()
  # remove non letters
  text = re.sub(r"[^a-zA-Z0-9]", " ", text)
  # tokenize
  words = text.split()
  # remove stopwords
  words = [w for w in words if w not in stopwords.words("english")]
  # apply stemming
  words = [PorterStemmer().stem(w) for w in words]
  # return list
  return words

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [55]:
df_annotated["lemmatizing_words"] = np.array([str(tweet_to_words(t)) for t in df_annotated["text"]])

In [ ]:
foundations = ['care','harm','fairness','cheating','loyalty','betrayal','authority','subversion', "liberty", "opression"]

In [63]:
df_annotated["text"][df_annotated["annotation"] =="Proportionality"]

7       What proportion of tipped workers don't want i...
17      It's not useless and the free market has *not*...
37      Bourdain was fighting with depression. I've be...
45      The court ruled that no reasonable viewer woul...
52      What surprises me even more is many of these f...
                              ...                        
4955    Wow some of them really are salty af. \n\n"Mac...
4965    Can you handle him at his Hollande? Because if...
4973    Introducing partial proportional voting to the...
4982    Not really, most sugar daddies are older marri...
4992    Yeah that was my point. Maybe Hillary is more ...
Name: text, Length: 472, dtype: object

In [62]:
df_annotated["text"].iloc[7]

"What proportion of tipped workers don't want it to change?"

In [56]:
df_annotated["annotation"].value_counts()

Care                                  973
Authority                             814
Equality                              618
Proportionality                       472
Loyalty                               306
                                     ... 
Purity,Authority,Equality               1
Care,Equality,Purity                    1
Care,Equality,Loyalty                   1
Loyalty,Equality,Proportionality        1
Proportionality,Equality,Authority      1
Name: annotation, Length: 96, dtype: int64

In [50]:
from gensim.models import Word2Vec

In [64]:
# For type hinting
from typing import List
def w2v_trainer(doc_tokens =  List[str], epochs: int = 10,
                workers: int = 3,
                vector_size: int = 300,
                window: int = 5,
                min_count: int = 2):
    """ 
    Going through a list of lists, where each list within the main list contains a set of tokens from a doc, this function trains a Word2Vec model,
    then creates two objects to store keyed vectors and keyed vocabs   
    Parameters:
    doc_tokens   : A tokenized document 
    epochs       : Number of epochs training over the corpus
    workers      : Number of processors (parallelization)
    vector_size  : Dimensionality of word embeddings
    window       : Context window for words during training
    min_count    : Ignore words that appear less than this
    Returns:
    keyed_vectors       : A word2vec vocabulary model
    keyed_vocab 
    
    """
    w2v_model = Word2Vec(doc_tokens,
                         epochs=10,
                         workers=3,
                         vector_size=300,
                         window=5,
                         min_count=2)
    
    # create objects to store keyed vectors and keyed vocabs
    keyed_vectors = w2v_model.wv
    keyed_vocab = keyed_vectors.key_to_index
    
    return keyed_vectors, keyed_vocab
    

In [68]:
word_vec, vocab_dict = w2v_trainer(df_comments["comment_important_words"].values())
print(word_vec)

KeyedVectors<vector_size=300, 41 keys>


In [69]:
df_comments["comment_important_words"]

0       ['adult', 'brother', 'syndrom', 'live', 'indep...
1       ['overthink', 'entir', 'situat', 'think', '99'...
2       ['mentor', 'adult', 'syndrom', 'spoke', 'well'...
3       ['harm', 'check', 'could', 'practic', 'crane',...
4       ['good', 'un', 'go', 'ask', 'fair', 'may', 'ac...
                              ...                        
7083    ['bro', 'made', 'mistak', 'sorri', 'perfect', ...
7084    ['ir', 'trip', 'think', 'report', 'gift', 'nam...
7085                          ['common', 'misunderstand']
7086    ['ye', 'also', 'sell', 'someon', 'market', 'wo...
7087                          ['properti', 'tax', 'work']
Name: comment_important_words, Length: 4699, dtype: object

In [72]:
word_vec[1]

array([-1.64622724e-01,  7.33440788e-03,  1.63626410e-02,  1.82186157e-01,
       -1.23987108e-01,  1.66091070e-01,  1.17222250e-01,  7.64761865e-03,
        4.86143259e-03, -2.79219039e-02, -3.34663019e-02, -1.07294910e-01,
       -6.36387989e-02, -1.03187762e-01,  6.89382330e-02,  5.89061491e-02,
       -2.74437070e-01,  7.30732456e-02,  1.14516497e-01, -5.68372458e-02,
        1.28345713e-01,  1.94325879e-01, -5.60992770e-02,  1.04937680e-01,
        1.95379797e-02, -3.14322934e-02,  4.51897353e-01, -2.38913566e-01,
       -1.97140217e-01,  5.65069765e-02, -1.86316416e-01,  3.12230103e-02,
        1.95791587e-01,  7.58144036e-02, -1.90628052e-01,  6.53116126e-03,
        2.90427476e-01,  1.74324811e-01, -4.37632427e-02,  2.53378332e-01,
       -2.22319774e-02,  1.61749739e-02, -1.92347139e-01,  1.01258829e-01,
       -2.13491648e-01,  5.18669784e-02,  7.94762895e-02,  1.73645467e-01,
        1.31996810e-01, -3.74557562e-02,  1.34735554e-02,  1.86693877e-01,
        4.03368287e-02, -

In [ ]:
from keras.layers import Input, UpSampling2D,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.models import Model
import keras.backend as K
from keras.callbacks import Callback
from keras.optimizers import SGD
from keras.datasets import imdb
from keras.utils import to_categorical
from keras.datasets import cifar10
import tensorflow as tf
from keras.metrics import categorical_accuracy, categorical_crossentropy
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
import numpy as np
import pickle, os, zipfile, glob

    

class PseudoCallback(Callback):

  def __init__(self, model, n_labeled_sample, batch_size, X, Y, X_unlabeled, n_classes, padding_len = 1000):

    #how many samples n labels to take at a time (streaming)
    self.n_labeled_sample = n_labeled_sample
    self.batch_size = batch_size
    self.model = model
    #how many classes in the data set
    self.n_classes = n_classes
    #this is to make sure evey vector has the same number of words
    self.maxlen =padding_len

    #given x and y (which are labeled), split the data into train and test
    X_train, self.X_test, y_train, self.y_test = train_test_split(X,Y,test_size=0.3, random_state=100, shuffle=True)
    
    #shuffle the indices for random order, that way lables arent lost
    indices = np.arange(X_train.shape[0])
    np.random.shuffle(indices)

    self.X_train_labeled = tf.keras.utils.pad_sequences(X_train[indices[:n_labeled_sample]], self.maxlen)
    self.y_train_labeled = y_train[indices[:n_labeled_sample]]

    #this one has the unlabeled variables we input
    self.X_train_unlabeled = tf.keras.utils.pad_sequences(X_unlabeled[indices[n_labeled_sample:]], self.maxlen)

    #this is to test the performance on the labeled data
    self.y_train = y_train[indices[n_labeled_sample:]]
    self.X_test = tf.keras.utils.pad_sequences(X_train, self.maxlen)

    #cant because now I will have many labels (10),if needed the values need tobe random vals similar to hot encoded labels
    #self.y_train_unlabeled_prediction = np.random.randint(2, size=(self.y_train_unlabeled_groundtruth.shape[0]))
    #self.y_train_unlabeled_prediction = np.random.choice(y_train, size=(self.y_train.shape[0]))
   
    self.train_steps_per_epoch = X_train.shape[0] // batch_size
    self.test_stepes_per_epoch = self.X_test.shape[0] // batch_size
    
    self.alpha_t = 0.0

    self.unlabeled_accuracy = []
    self.labeled_accuracy = []

  def train_mixture(self):
    
    X_train_join = np.r_[self.X_train_labeled, self.X_train_unlabeled]
    y_train_join = np.r_[self.y_train_labeled, self.y_train_unlabeled_prediction]
    flag_join = np.r_[np.repeat(0.0, self.X_train_labeled.shape[0]),
                      np.repeat(1.0, self.X_train_unlabeled.shape[0])].reshape(-1,1)
    indices = np.arange(flag_join.shape[0])
    np.random.shuffle(indices)
    return X_train_join[indices], y_train_join[indices], flag_join[indices]

  def train_generator(self):
    while True:
        X, y, flag = self.train_mixture()
        n_batch = X.shape[0] // self.batch_size
        for i in range(n_batch):
            X_batch = (X[i*self.batch_size:(i+1)*self.batch_size]).astype(np.float32)
            y_batch = to_categorical(y[i*self.batch_size:(i+1)*self.batch_size], self.n_classes)
            y_batch = np.c_[y_batch, flag[i*self.batch_size:(i+1)*self.batch_size]]
            yield X_batch, y_batch

  def test_generator(self):
      while True:
          indices = np.arange(self.y_test.shape[0])
          np.random.shuffle(indices)
          for i in range(len(indices)//self.batch_size):
              current_indices = indices[i*self.batch_size:(i+1)*self.batch_size]
              X_batch = (self.X_test[current_indices]).astype((np.float32))
              
              #(np.object)
              y_batch = to_categorical(self.y_test[current_indices], self.n_classes)
              y_batch = np.c_[y_batch, np.repeat(0.0, y_batch.shape[0])] 
              yield X_batch, y_batch

  def loss_function(self, y_true, y_pred):
      y_true_item = y_true[:, :self.n_classes]
      unlabeled_flag = y_true[:, self.n_classes]
      entropies = categorical_crossentropy(y_true_item, y_pred)
      coefs = 1.0-unlabeled_flag + self.alpha_t * unlabeled_flag # 1 if labeled, else alpha_t
      return coefs * entropies

  def accuracy(self, y_true, y_pred):
      y_true_item = y_true[:, :self.n_classes]
      return categorical_accuracy(y_true_item, y_pred)

  def on_epoch_end(self, epoch, logs):
      #defining alpha
      if epoch < 10:
          self.alpha_t = 0.0
      elif epoch >= 70:
          self.alpha_t = 3.0
      else:
          self.alpha_t = (epoch - 10.0) / (70.0-10.0) * 3.0
    
      self.y_train_unlabeled_prediction = np.argmax(
          self.model.predict(self.X_train_unlabeled), axis=-1,) 
      y_train_labeled_prediction = np.argmax(
          self.model.predict(self.X_train_labeled), axis=-1)

      self.unlabeled_accuracy.append(np.mean(
          self.y_train_unlabeled_groundtruth == self.y_train_unlabeled_prediction))
      self.labeled_accuracy.append(np.mean(
          self.y_train_labeled == y_train_labeled_prediction))
      print(f"labeled accuracy {self.labeled_accuracy[-1]}, unlabeled accuracy : {self.unlabeled_accuracy[-1]}")

  def on_train_end(self, logs):
      y_true = np.ravel(self.y_test)
      emb_model = Model(self.model.input, self.model.layers[-2].output)
      embedding = emb_model.predict(self.X_test )
      proj = TSNE(n_components=2).fit_transform(embedding)
      cmp = plt.get_cmap("tab10")
      plt.figure()
      for i in range(10):
          select_flag = y_true == i
          plt_latent = proj[select_flag, :]
          plt.scatter(plt_latent[:,0], plt_latent[:,1], color=cmp(i), marker=".")
      plt.savefig(f"result_pseudo_trans_mobile/embedding_{self.n_labeled_sample:05}.png")



In [ ]:
foundations = ['care','harm','fairness','cheating','loyalty','betrayal','authority','subversion','sanctity','degradation']

virtues = ['care','fairness','loyalty','authority','sanctity']
vices = ['harm','cheating','betrayal','subversion','degradation']

base_f = ['care_p','fairness_p','loyalty_p','authority_p','sanctity_p']
sents = ['care_sent','fairness_sent','loyalty_sent','authority_sent','sanctity_sent']